<a href="https://colab.research.google.com/github/Elzfe09/RAG-ChatBot/blob/main/RAG_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install langchain langchain_core langchain_openai langchain-community langchain_text_splitters openai pypdf python-dotenv numpy scikit-learn matplotlib scikit-learn scipy beautifulsoup4 pypdf chroma langgraph typing_extensions

  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of langgraph-prebuilt to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langgraph-prebuilt to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.6/326.6 kB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━

In [ ]:
import tiktoken
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import OpenAIEmbeddings

In [ ]:
load_dotenv('template.env')

True

In [ ]:
trace = os.getenv('LANGCHAIN_TRACING_V2')
langsmith = os.getenv('LANGCHAIN_API_KEY')

gpt = ChatOpenAI(
    model = 'gpt-4',
    openai_api_key = os.getenv('OPENAI_API_KEY'),
    temperature = 0
)

embeddings1 = OpenAIEmbeddings(
    openai_api_key = os.getenv('OPENAI_API_KEY'),
    model = 'text-embedding-3-large'
)

embeddings2 = OpenAIEmbeddings(
    model = 'text-embedding-3-large',
    openai_api_key = os.getenv('OPENAI_API_KEY'),
    dimensions = 1024
)

In [81]:
#load pdf

from langchain_community.document_loaders import PyPDFLoader
file_path = '/content/TravelEasy Pre-Ex Policy QTD032212-PX.pdf'
loader= PyPDFLoader(file_path)

docs = loader.load()

print(len(docs))

for i in docs[:4]: #take page 1-4
  print(i)
  print('------------------------------------------------------------------')

59
page_content='MSIG Insurance (Singapore) Pte. Ltd. (Co. Reg. No. 200412212G) 
4 Shenton Way, #21-01, SGX Centre 2, Singapore 068807 
Tel +65 6827 7888 
msig.com.sg 
 
                                                                                        QTD032212-PX | Page 1 of 59 
TRAVELEASY®/TRAVELEASY® PRE-EX 
INSURANCE POLICY  
 
This is your TravelEasy® or TravelEasy® Pre-Ex insurance policy document. It is important that you read this policy document 
together with the schedule and any amendment or endorsement issued to make sure that you have the protection you 
need and to avoid any misunderstanding. 
 
If there are any changes that may affect the insurance provided, please let us know immediately. 
 
We suggest that you let your family members know about this insurance cover as it would be helpful if you or they need to 
make a claim. 
 
IMPORTANT NOTICE 
The insurance cover provided to you under this policy is based on the information you have given us. You must tell us a

In [82]:
# count token

def count_tokens(text, model_name = 'text-embedding-ada-002'):
  encoding = tiktoken.encoding_for_model(model_name)
  tokens = encoding.encode(text)
  token_count = len(tokens)
  return token_count

document= docs[0].page_content

#checck token count
token_limit = 8192
token_count = count_tokens(document)
print(f'token count: {token_count}')

#split into diff chunk
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
splits = text_splitter.split_documents(docs)
#store embedding in vectorstore docs
vector_store = InMemoryVectorStore.from_documents(
    documents = splits,
    embedding = embeddings2
)
retriever = vector_store.as_retriever()

token count: 427


In [99]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

#add prompt
system_prompt = (
    'you are a support-assistance task for answering question from customer that want to ask about insurance policy before purchase'
    'give the answer very relevant, concise and clear pointing to the core question'
    'wrapped in maximum 1 paragraph'
    'use this pieces of retrieved content and relevant available document to answer'
    '\n\n{context}' # Add context variable here

)
prompt = ChatPromptTemplate.from_messages(
    [
        ('system',system_prompt),
        ('human', '{input}'),
    ]
)
#combine prompt with our model
qna_chain = create_stuff_documents_chain(gpt, prompt)
#match retriever (vector db store) with qs chain
rag_chain = create_retrieval_chain(retriever, qna_chain)

In [106]:
results = rag_chain.batch([{
    'input':'summarized in one sentences, what things only covered in Premier Plan?',
    'chat_history':[]
}])

for result in results:
  print(result['context'][0].page_content)

QTD032212-PX | Page 55 of 59 
Benefits 
Standard Plan Elite Plan Premier Plan 
Individual 
Adult & 
children 
Family 
cover 
(in total)1 
Individual 
Adult & 
children 
Family 
cover 
(in total)1 
Individual 
Adult & 
children 
Family 
cover 
(in total)1 
Limit of Benefits 
Section 40 Legal expenses for wrongful arrest or detention 
Adult Not covered $5,000 $5,000 $10,000 $10,000 Child $5,000 $10,000 
Lifestyle cover 
Section 41 Adventurous activities 
cover Not covered Yes Yes 
Section 42 Golfer's cover    
Damage or loss of golf equipment 
Not covered 
$1,000 $1,500 
Unused green fees due to injury or 
illness $500 $750 
Hole-in-one $500 $750 
Section 43 Unused entertainment 
ticket  Not covered $300 $600 $500 $1,000 
Section 44 Rental vehicle excess Not covered $1,000 $1,500 
Section 45 Returning a rental 
vehicle Not covered $500 $750 
Section 46 Home contents Not covered $10,000 $15,000 
Section 47 Domestic pets care Not covered $50 per day 
Max. $500 
$75 per day 
Max. $750


## Adding chat history

user queries may require conversational context to understand

example:
User: may i know what things not covered in insurances

AI: it's.............

User: why like that?


In [101]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import create_history_aware_retriever

contextualise_q_system_prompt = (
    'given a chat history and the latest user question'
    'which might reference context in the chat history'
    'formulate a standalone question which can be understood'
    'without the chat history, do not answer the question'
    'just reformulate it if needed and otherwise return as it is '
)
#update prompt by integrate history input

contextualise_q_prompt=  ChatPromptTemplate.from_messages(
    [
        ('system', contextualise_q_system_prompt),
        MessagesPlaceholder('chat_history'),
        ('human','{input}')
    ]
)

history_retriever = create_history_aware_retriever(
    gpt, retriever, contextualise_q_prompt
)


In [102]:
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ('system', system_prompt),
        MessagesPlaceholder('chat_history'),
        ('human','{input}'),
    ]
)
qna_chain = create_stuff_documents_chain(gpt, qa_prompt)
rag_chain = create_retrieval_chain(history_retriever, qna_chain)

In [103]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = []
question = 'is this insurance cover a wedding equipment?'
msg1 = rag_chain.invoke({
    'input': question,
    'chat_history': chat_history
})
chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content = msg1['answer']),
    ]
)

second_qs = 'how does it works?'
msg2 = rag_chain.invoke({
    'input': second_qs,
    'chat_history': chat_history
})

print(msg2['answer'])

The insurance works by providing coverage for accidental loss or damage to wedding-related items such as bridal and ceremonial clothing, wedding rings, jewellery, and accessories during the journey. These items must be transported as carry-on baggage on public transport, as they are not covered if transported as checked-in baggage. If a loss or damage occurs, you must first claim from the relevant parties such as public transport, accommodation, or service provider. If they deny compensation, you can then submit a claim to the insurance company, providing proof of denial from the relevant parties. The maximum payout depends on the plan you choose, with the Premier Plan offering the highest coverage. However, the Standard Plan does not cover these items. Please note that this coverage does not apply to insured children.


# Langgraph

In [107]:
from typing import Sequence

from langchain_core.messages import BaseMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, StateGraph
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict

In [116]:
class State(TypedDict):
  input: str
  chat_history: Annotated[Sequence[BaseMessage],add_messages]
  context: str
  answer: str

#define a node that runs the rag chain, update chat history with input message and response
def call_model(state: State):
  response = rag_chain.invoke(state)
  return {
      'chat_history': [
          HumanMessage(state['input']),
          AIMessage(response['answer']),
      ],
      'context': response['context'],
      'answer': response['answer'],
  }

  #graph building
  workflow = StateGraph(state_schema = State)
  workflow.add_edge(START, 'model')
  workflow.add_node('model', call_model)

  #compile
  memory = MemorySaver()
  app = workflow.compile(checkpointer = memory)

## Config Dictionary

manage multiple conversation threads within the application. assign a unique id identifier number to distinguish between user individual chat and track separate conversation context meaning, handle multiple user or ongoing threads independently

In [119]:
#setting unique personnel chat
config = {'configurable':{'thread_id':'abc123'}}
result = app.invoke(
    {'input': 'what is the difference between premier and standard plan?'},
    config = config
)
print(result.keys())
print(result['input']) #user question
print(result['chat_history'])
print(result['context'])
print(result['answer']) #our build rag answer

dict_keys(['input', 'chat_history', 'context', 'answer'])
what is the difference between premier and standard plan?
[HumanMessage(content='what is the difference between premier and standard plan?', additional_kwargs={}, response_metadata={}, id='8320683c-1c80-483a-a4c6-04a4092ac607'), AIMessage(content="The Premier and Standard plans differ in terms of coverage limits and benefits. The Premier Plan offers higher coverage limits, for example, $7,500 for each insured person and $15,000 in total for adult and children cover or family cover, compared to the Standard Plan which offers $3,000 for each insured person and $6,000 in total for adult and children cover or family cover. The Premier Plan also provides more benefits such as legal expenses for wrongful arrest or detention, golfer's cover, home contents cover, and higher limits for traditional Chinese medicine expenses, maternity medical expenses overseas, overseas hospitalisation daily benefit, insolvency of licensed travel operator

In [121]:
result = app.invoke(
    {'input':'which one both of them preffered for family of 3 without any medical past issues and normal condition?'},
    config = config,
)
print(result['answer'])

For a family of three with no past medical issues and normal conditions, the Premier Plan could be a more suitable choice. It offers higher coverage limits and more comprehensive benefits, including higher limits for traditional Chinese medicine expenses, maternity medical expenses overseas, and overseas hospitalisation daily benefit. It also covers legal expenses for wrongful arrest or detention, golfer's cover, home contents, and more. These extensive coverages can provide a more secure protection for your family during the journey. However, the final decision should also consider your family's specific needs, travel plans, and budget.


In [122]:
chat_history = app.get_state(config).values['chat_history']
for message in chat_history:
  message.pretty_print()

  #track the past chat

================================ Human Message =================================

what is the difference between premier and standard plan?
================================== Ai Message ==================================

The Premier and Standard plans differ in terms of coverage limits and benefits. The Premier Plan offers higher coverage limits, for example, $7,500 for each insured person and $15,000 in total for adult and children cover or family cover, compared to the Standard Plan which offers $3,000 for each insured person and $6,000 in total for adult and children cover or family cover. The Premier Plan also provides more benefits such as legal expenses for wrongful arrest or detention, golfer's cover, home contents cover, and higher limits for traditional Chinese medicine expenses, maternity medical expenses overseas, overseas hospitalisation daily benefit, insolvency of licensed travel operator, travel cancellation, and travel postponement among others. The Standard Plan does 

## Agent

leverage the reasoning capabilities of LLMs to make decisions during execution. agent allow you to offload some discretion over retrieval process they give less predictable behavior than chains, offer some advantages:

- agent send input to retriever directly without needing to build in contextualization as we did above:
- agents execute multiple retrieval steps or refrain from executing a retrieval step altogether



In [126]:
from langchain.tools.retriever import create_retriever_tool
#create tool for agent to use

tool = create_retriever_tool(
    retriever,
    'blog_post_retriever', #agent name
    'searches and return excerpts from the autonomous agents blog post'
)
tools = [tool]

tool.invoke('what the benefit that differ for usual occupation between doctor/dentist?')

'Dentist A legally registered dental practitioner qualified to practise in line with the laws of \nthe country in which the practice is granted. The dentist must not be an insured \nperson, an insured personfamily member, travel companion, a business partner, \nan employee or employer of an insured person, or a person related to an insured \nperson in any way. \nDoctor A legally registered medical practitioner qualified to practise western medicine and \nsurgery in line with the laws of the country in which the practice is granted. The \ndoctor must not be an insured person, an insured personfamily member, travel \ncompanion, a business partner, an employee or employer of an insured person, or a \nperson related to an insured person in any way. \nEntertainment ticket A ticket granting admission to theme parks, musicals, plays, theatre or drama \nperformances, concerts and sports events but excludes movie tickets.\n\nsection. For any claim under Pre-Ex Standard, Pre-Ex Elite or Pre-Ex P

## Agent Constructor using Langgraph

In [138]:

agent_executor = create_react_agent(gpt,tools)
#combine gpt model and retrieval tool

query = 'does the insurancee cover funeral for unforeseen circumstances such as death during trip?'

for event in agent_executor.stream(
    {'messages':[HumanMessage(content ='Hi i want to ask question for my trip inssurance' )]},
    config=config,
    stream_mode = 'values'
):
    event['messages'][-1].pretty_print()

for event in agent_executor.stream(
    {'messages': [HumanMessage(content=  query)]},
    confif = config,
    stream_mode = 'values'
):
    event['messages'][-1].pretty_print()

/tmp/ipython-input-3992379015.py:1: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent_executor = create_react_agent(gpt,tools)


================================ Human Message =================================

Hi i want to ask question for my trip inssurance
================================== Ai Message ==================================

Sure, I'd be happy to help with your trip insurance questions. What would you like to know?
================================ Human Message =================================

does the insurancee cover funeral for unforeseen circumstances such as death during trip?
================================== Ai Message ==================================
Tool Calls:
  blog_post_retriever (call_zgjVAygXxZC2dBuOVM4zUYqa)
 Call ID: call_zgjVAygXxZC2dBuOVM4zUYqa
  Args:
    query: insurance coverage for funeral in case of death during trip
================================= Tool Message =================================
Name: blog_post_retriever

SECTION 3  FUNERAL EXPENSES DUE TO ACCIDENTAL DEATH 
 
If the insured person suffers an injury during the journey outside Singapore which, within 12 